In [3]:
import main as m
import time
import osmnx as ox
import importlib as i
from shapely import wkt
import folium
from folium.plugins import MarkerCluster

#isnull

In [8]:
PATH = {}
PATH["rawDATA"] = "data/unprocessed/"
t = "tourism"
keys = ["attraction", "artwork", "gallery", "museum", "viewpoint", "yes"]

# https://wiki.openstreetmap.org/wiki/Key:historic
t = "historic"
keys = ["memorial", "monument"]
# memorial gives timeout https://wiki.openstreetmap.org/wiki/Key:historic
keys = ["monument"]

# timeout? https://github.com/gboeing/osmnx/pull/492

In [9]:
# check
place = "Hungary"
for k in keys:
    searchTerm = str("'"+t+"': '"+k+"'")
    print(str("'"+t+"': '"+k+"'"))
    start = time.time()
    tags = {searchTerm}
    #print({t:k})
    raw = ox.geometries_from_place(place, tags={t:k})
    display(raw.explore())
    try:
        raw.to_parquet(PATH["rawDATA"]+place+"-"+k+".parquet")
    except:
        raw.to_csv(PATH["rawDATA"]+place+"-"+k+".csv")  
    end = time.time()
    print(end - start)

'historic': 'monument'


ReadTimeout: HTTPSConnectionPool(host='overpass-api.de', port=443): Read timed out. (read timeout=180)

In [3]:
# read in 
locations = {}
place = "Hungary"
for k in keys:
    print(k)
    try:
        locations[k] = m.gpd.read_parquet(PATH["rawDATA"]+place+"-"+k+".parquet")
    except:
        locations[k] = m.pd.read_csv(PATH["rawDATA"]+place+"-"+k+".csv", low_memory=False)
        locations[k]["geometry"] = locations[k]["geometry"].apply(wkt.loads)
        locations[k] = m.gpd.GeoDataFrame(locations[k], geometry = locations[k]["geometry"], crs = 'EPSG:4326')
        #t = m.gpd.GeoDataFrame(locations[k], geometry = m.gpd.points_from_xy(locations[k]['x'], locations[k]['y']), crs = 'EPSG:4326')
        #locations[k].crs = "epsg:4326"
        #t = m.pd.read_csv(PATH["rawDATA"]+k+".csv")
        #locations[k] = m.gpd.GeoDataFrame(t, geometry=t.geometry, crs = 'EPSG:4326')

attraction
artwork
gallery
museum
viewpoint
yes


In [4]:
colors = [
    'red',
    'blue',
    'gray',
    'darkred',
    'lightred',
    'orange',
    'beige',
    'green',
    'darkgreen',
    'lightgreen',
    'darkblue',
    'lightblue',
    'purple',
    'darkpurple',
    'pink',
    'cadetblue',
    'lightgray',
    'black'
]

icons = ["magnet", "hammer", "palette", "building-columns","eye","question"]
# icons list
# https://fontawesome.com/search?q=caravan&o=r

ma = folium.Map(location=[47.1611615, 19.5057541], zoom_start=7, tiles='OpenStreetMap')
marker_cluster = MarkerCluster().add_to(ma)
c = 0
l = list(locations.keys())
WHERE = "Magyarország"
for i in l:
    locations[i]["lat"] = locations[i].geometry.centroid.y
    locations[i]["lon"] = locations[i].geometry.centroid.x
    locations[i]["websiteBool"] = locations[i]["website"].apply(lambda x: 1 if m.pd.isnull(x) == True else 0)
    print(i)
    #print(locations[i].columns.values.tolist())
    for _, r in locations[i].iterrows():
        #m.folium.Marker(location=[r['lat'], r['lon']], popup='length: {} <br> area: {}'.format(r['Shape_Leng'], r['Shape_Area'])).add_to(m)
        #folium.Marker(location=[r['lat'], r['lon']], popup=r["name"], icon=folium.Icon(color=colors[c], prefix='fa', icon=icons[c])).add_to(ma)
        #link = str("<a href=https://www.google.com/search?q="+WHERE+" "+r["name"]+" target=_blank >"+popup=r["name"]+ "</a>")
        linkGog = str(WHERE+" "+str(r["name"])+" "+str(r["closestCity"]))
        linkGog = linkGog.replace(" ", "+")
        if r["name"] == None:
            linkName = "Link"
        else:
            linkName = r["name"]
            
        if r["websiteBool"] == 0: # if it is 0, means there is a website
            #print(r["website"])
            link = str("<a href="+str(r["website"])+" target=_blank > "+ str(linkName)+ "</a>")
        else:
            #print("wrong link", r["website"])
            link = str("<a href=https://www.google.com/search?q="+linkGog+" target=_blank > "+ str(linkName)+ "</a>")
        #link = str("<a href=https://www.google.com/search?q="+WHERE+" "+str(r["name"])+" target=_blank > "+str(r["name"])+ "</a>")
        #print(link)
        folium.Marker(location=[r['lat'], r['lon']], 
                      popup=link, 
                      icon=folium.Icon(color=colors[c], prefix='fa', icon=icons[c])).add_to(marker_cluster)
    c += 1
#ma
ma.save("Hungary.html")

C:\Users\MUSIC\AppData\Local\Temp\ipykernel_10144\3285926332.py:32: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  locations[i]["lat"] = locations[i].geometry.centroid.y
C:\Users\MUSIC\AppData\Local\Temp\ipykernel_10144\3285926332.py:33: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  locations[i]["lon"] = locations[i].geometry.centroid.x


attraction


C:\Users\MUSIC\AppData\Local\Temp\ipykernel_10144\3285926332.py:32: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  locations[i]["lat"] = locations[i].geometry.centroid.y
C:\Users\MUSIC\AppData\Local\Temp\ipykernel_10144\3285926332.py:33: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  locations[i]["lon"] = locations[i].geometry.centroid.x


artwork


C:\Users\MUSIC\AppData\Local\Temp\ipykernel_10144\3285926332.py:32: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  locations[i]["lat"] = locations[i].geometry.centroid.y
C:\Users\MUSIC\AppData\Local\Temp\ipykernel_10144\3285926332.py:33: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  locations[i]["lon"] = locations[i].geometry.centroid.x
C:\Users\MUSIC\AppData\Local\Temp\ipykernel_10144\3285926332.py:32: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  locations[i]["lat"] = locations[i].geometry.centroid.y
C:\Users\MUSIC\AppData\Local\Temp\ipykernel_10144\3285926332.py:33: Us

gallery
museum


C:\Users\MUSIC\AppData\Local\Temp\ipykernel_10144\3285926332.py:32: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  locations[i]["lat"] = locations[i].geometry.centroid.y
C:\Users\MUSIC\AppData\Local\Temp\ipykernel_10144\3285926332.py:33: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  locations[i]["lon"] = locations[i].geometry.centroid.x


viewpoint


C:\Users\MUSIC\AppData\Local\Temp\ipykernel_10144\3285926332.py:32: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  locations[i]["lat"] = locations[i].geometry.centroid.y
C:\Users\MUSIC\AppData\Local\Temp\ipykernel_10144\3285926332.py:33: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  locations[i]["lon"] = locations[i].geometry.centroid.x


yes


In [8]:
ma.save("Hungary.html")

In [58]:
import pandas as pd